In [ ]:
import math
import sys

sys.path.append("../")

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

np.random.seed(1)

def custom_save(function=plt.savefig, fname=""):
    if not os.path.exists(os.path.dirname(fname)):
        os.makedirs(os.path.dirname(fname))
        print('created new directory', os.path.dirname(fname))
    try:
        # to make sure suptitle is not cut off
        function(fname, bbox_inches="tight")
    except:
        function(fname)
    print("saved as", fname)

from matplotlib import rcParams

rcParams["figure.max_open_warning"] = False

In [ ]:
def read_csv(fname):
    def convert(row):
        arrays = ["signals_real_vect", "signals_imag_vect", "frequencies"]
        ints = ["n_mics", "n_frequencies", "timestamp"]
        for array_name in arrays:
            row[array_name] = np.fromstring(row[array_name].replace('[', '').replace(']',''), sep=' ', dtype=np.float32)
        for int_name in ints:
            row[int_name] = int(row[int_name])
        return row
    df = pd.read_csv(fname)
    df = df.apply(convert, axis=1)
    return df

def get_spectrogram(df):
    spectrogram = None
    for i, row in df.iterrows(): 
        signals_f = row.signals_real_vect + 1j * row.signals_imag_vect
        signals_f = signals_f.reshape((row.n_mics, row.n_frequencies))
        new_column = np.sum(np.abs(signals_f), axis=0)

        #signals_f = signals_f.reshape((int(row["n_frequencies"]), int(row["n_mics"])))
        #new_column = np.sum(np.abs(signals_f), axis=1)

        if spectrogram is None:
            spectrogram = new_column
        else:
            spectrogram = np.c_[spectrogram, new_column]
    return spectrogram

In [ ]:
dirname = '../experiments/2020_09_17_white-noise-static/csv_files'
fname_source = dirname + '/nomotors_nosnr_noprops_source_45.csv'
fname_all =  dirname + '/motors_nosnr_noprops_source_45.csv'
#fname_props = dirname + '../motors_nosnr_noprops_nosource_45.csv'
fname_props = dirname + '/motors_nosnr_noprops_nosource.csv'

df_source = read_csv(fname_source)
spectrogram_source = get_spectrogram(df_source)

plt.figure()
plt.pcolormesh(spectrogram_source)
plt.xlabel('time index [-]')
plt.ylabel('frequency index [-]')
plt.title('source')

df_all = read_csv(fname_all)
spectrogram_all = get_spectrogram(df_all)

plt.figure()
plt.pcolormesh(spectrogram_all)
plt.xlabel('time index [-]')
plt.ylabel('frequency index [-]')
plt.title('all')

df_props = read_csv(fname_props)
spectrogram_props = get_spectrogram(df_props)

plt.figure()
plt.pcolormesh(spectrogram_props)
plt.xlabel('time index [-]')
plt.ylabel('frequency index [-]')
plt.title('props')

In [ ]:
duration = 30 * 1e3 # miliseconds

# create sound level plot over time
sound_level_source = np.sum(spectrogram_source, axis=0) / spectrogram_source.shape[0]
sound_level_all = np.sum(spectrogram_all, axis=0) / spectrogram_all.shape[0]
sound_level_props = np.sum(spectrogram_props, axis=0) / spectrogram_props.shape[0]

# detect the end time and cut fixed time before it
end_index_source = np.where(sound_level_source > 1e-2)[0][-1]
end_time_source = df_source.timestamp[end_index_source]
plt.semilogy(df_source.timestamp - end_time_source, sound_level_source, color='C0', label='source')

end_index_all = np.where(sound_level_all > 1e-2)[0][-1]
end_time_all = df_all.timestamp[end_index_all]
plt.semilogy(df_all.timestamp - end_time_all, sound_level_all, color='C1', label='all')

end_index_props = np.where(sound_level_props > 1e-2)[0][-1]
end_time_props = df_props.timestamp[end_index_props]
plt.semilogy(df_props.timestamp - end_time_props, sound_level_props, color='C2', label='props')

plt.xlabel('time [ms]')
plt.ylabel('sound level')
plt.legend()

In [ ]:
# calculate spatial spectrum using mvdr beamforming method 

df_short = df_source.loc[(end_time_source - df_source.timestamp) < duration]